In [13]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn

model = "/Volumes/Seagate/Filen/data/public/models/company_reviews/10000_samples_best_phobert_sentiment.bin"


class SentimentClassifier(nn.Module):
    def __init__(self, n_classes, bert_model):
        super().__init__()
        self.phobert = bert_model
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.phobert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        output = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = output.last_hidden_state.mean(dim=1)
        return self.out(self.drop(pooled))


tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
bert_model = AutoModel.from_pretrained("vinai/phobert-base-v2")
classifier = SentimentClassifier(n_classes=3, bert_model=bert_model).to("cpu")
classifier.load_state_dict(torch.load(model))

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
def encode(review):
    encoding = tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_review = {
        "input_ids": encoding["input_ids"],
        "attention_mask": encoding["attention_mask"],
    }

    return input_review


classifier.eval()
classes = {
    0: "Negative",
    1: "Neutral",
    2: "Positive",
}

In [64]:
review = [
    "Ủa chừng nào có lương?",
    "Ở đây nhiều đồ ăn",
    "Đồng nghiệp không ổn định",
    "Làm ở đây cũng tạm OK",
    "Ahaha mắc cười ghê",
    "Ahihi tưởng thế nào",
]
input_review = [encode(review) for review in review]

for each_input in input_review:
    with torch.no_grad():
        logits = classifier(**each_input).cpu().numpy()
        predicted_class = logits.argmax(axis=1)[0]
        print(f"Predicted class: {classes[predicted_class]}")


Predicted class: Negative
Predicted class: Positive
Predicted class: Negative
Predicted class: Positive
Predicted class: Negative
Predicted class: Negative
